In [43]:
import numpy as np
from sklearn.model_selection import train_test_split


In [44]:
import os
import cv2

try:
    os.remove("dataset/README")
except:
    print("README not found")

README not found


In [58]:
# pip install natsort 
from natsort import natsorted

all_images = []
all_labels = []

entries = os.listdir("dataset")
# Natsort is used to sort the images in the order of their names which the normal sort fails to do
entries = natsorted(entries)
for i,entry in enumerate(entries):
    images = os.listdir("dataset/"+entry)
    images = natsorted(images)
    for image in images:
        # Reading the image and converting it to grayscale
        img = cv2.imread("dataset/"+entry+"/"+image,cv2.IMREAD_GRAYSCALE)
        # Flatten / normalize the image
        all_images.append(img.flatten()/255)
        all_labels.append(i+1)

# Generating Data matrix
all_images = np.array(all_images)
all_labels = np.array(all_labels)
all_labels=all_labels.reshape(all_labels.shape[0],-1)

print(f"Images shape: {all_images.shape}")
print(f"Labels shape: {all_labels.shape}")

Images shape: (400, 10304)
Labels shape: (400, 1)


In [59]:
# Splitting the data into train and test (70% for training and 30% images for testing)
X = all_images
Y = all_labels
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.3)

print(f"Xtrain shape: {Xtrain.shape}")
print(f"Ytrain shape: {Ytrain.shape}")
print(f"Xtest shape: {Xtest.shape}")
print(f"Ytest shape: {Ytest.shape}")

Xtrain shape: (280, 10304)
Ytrain shape: (280, 1)
Xtest shape: (120, 10304)
Ytest shape: (120, 1)


In [60]:
Xtrain_transposed = Xtrain.transpose()
means = [np.mean(row) for row in Xtrain_transposed]
means = np.array(means)
print(f"Means shape: {means.shape}")

Means shape: (10304,)


In [61]:
Z = Xtrain - means
print(f"Z shape: {Z.shape}")
cov = 1/len(Xtrain) * np.matmul(Z.transpose(),Z)
print(f"Covariance shape: {cov.shape}")

Z shape: (280, 10304)
Covariance shape: (10304, 10304)


In [62]:
eig_values, eig_vectors = np.linalg.eigh(cov)
idx = eig_values.argsort()[::-1]  
eig_values = eig_values[idx]
eig_vectors = eig_vectors[:,idx]

In [63]:
def get_dimensions(eigen_values, alpha):
    total = eig_values.sum()
    sum=0
    for i,val in enumerate(eigen_values):
        sum+=val
        if sum/total >= alpha:
            break
    return i+1

In [64]:
alphas = [0.8,0.85,0.9,0.95]
r = [get_dimensions(eig_values,alpha) for alpha in alphas]
r

[40, 59, 91, 148]

In [65]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report , accuracy_score
from sklearn.model_selection import GridSearchCV
p_train = []
p_test = []

for dim in (r):
    
    p_train_data = np.matmul(eig_vectors[:, 0:dim].T, Xtrain.T).T
    p_test_data = np.matmul(eig_vectors[:, 0:dim].T, Xtest.T).T

    p_train.append(p_train_data)
    p_test.append(p_test_data)

In [66]:
from sklearn.metrics import make_scorer

knn = KNeighborsClassifier()
parameters = {"n_neighbors":list(range(1,10,2))}
scorer = make_scorer(accuracy_score)
grid_search = GridSearchCV(knn,param_grid=parameters,cv=5,return_train_score=True,scoring=scorer)

In [67]:
preds = []
for i in range(len(p_train)):
    grid_search.fit(p_train[i],Ytrain.ravel())
    print(f"For alpha = {alphas[i]}, best parameters are {grid_search.best_params_} and best score is {grid_search.best_score_}")
    y_pred = grid_search.best_estimator_.predict(p_test[i])
    preds.append(y_pred)

c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


For alpha = 0.8, best parameters are {'n_neighbors': 1} and best score is 0.9428571428571428


c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


For alpha = 0.85, best parameters are {'n_neighbors': 1} and best score is 0.9464285714285715


c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


For alpha = 0.9, best parameters are {'n_neighbors': 1} and best score is 0.9464285714285714


c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


For alpha = 0.95, best parameters are {'n_neighbors': 1} and best score is 0.9392857142857143


In [68]:
for i in range(len(preds)):
    print(f"Test Accuracy for alpha = {alphas[i]} is {accuracy_score(Ytest.ravel(),preds[i])}")

Test Accuracy for alpha = 0.8 is 0.975
Test Accuracy for alpha = 0.85 is 0.9666666666666667
Test Accuracy for alpha = 0.9 is 0.9666666666666667
Test Accuracy for alpha = 0.95 is 0.975


In [71]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

for x in range(len(r)):
    
    p_train_data = np.matmul(eig_vectors[:, 0:r[x]].T, Xtrain.T).T
    p_test_data = np.matmul(eig_vectors[:, 0:r[x]].T, Xtest.T).T

    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(p_train_data, Ytrain)

    y_pred=neigh.predict(p_test_data)

    print('accuracy for alpha with number -->',alphas[x])
    print(classification_report(Ytest, y_pred))

accuracy for alpha with number --> 0.8
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      0.80      0.89         5
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         4
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         6
          15       0.80      1.00      0.89         4
          16       1.00      1.00      1.00         3
          17       1.00      1.00      1.0

c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\carin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\carin\AppData\Local\Pr